In [29]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [30]:
import pandas as pd

df = pd.read_excel('LinkedIn100Reviews.xlsx')


In [31]:
load_dotenv()

api_key  = os.environ['OPENAI_API_KEY']
# API configuration
openai.api_key = os.getenv(api_key)

In [32]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

In [33]:
params = set_open_params(temperature=2)

In [34]:
import openpyxl
# Create a new column 'Sentiment' in the DataFrame
df['review'] = ''

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the review text from the 'review' column
    review = row['review']
    
    # Create the prompt for classification
    prompt = f"Classify the text into satisfied, dissatisfied.\n\nReview: {review}"
    
    # Create the messages for the completion
    messages = [
                {
            "role": "system",
            "content": "You are an assistant that helps to classify reviews into satisfied, dissatisfied. The review is about a product or service. Classify the review into satisfied, dissatisfied."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    # Get the completion response
    response = get_completion(params, messages)
    
    # Get the classified sentiment from the response
    sentiment = response.choices[0].message.content
    
    # Update the 'Sentiment' column in the DataFrame
    df.at[index, 'Sentiment'] = sentiment

# Save the DataFrame to the Excel file
df.to_excel('LinkedIn100Reviews_Classified.xlsx', index=False)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}